In [ ]:
#default_exp models.baseline

In [ ]:
#all_func

# Baseline Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from peptide.preprocessing.data import *
from peptide.preprocessing.embedding import *

Load data

In [ ]:
merged_train_df, merged_test_df = get_all_data(merge=True)
acp_train_df, acp_test_df, amp_train_df, amp_test_df, dna_bind_train_df, dna_bind_test_df = get_all_data()

In [ ]:
acp_train_df.head(5)

,sequence,label_acp
0,RRWWRRWRRW,0
1,GWKSVFRKAKKVGKTVGGLALDHYLG,0
2,ALWKTMLKKLGTMALHAGKAALGAAADTISQGTQ,1
3,GLFDVIKKVAAVIGGL,1
4,VAKLLAKLAKKVL,1


In [ ]:
acp_test_df.head(5)

,sequence,label_acp
0,FLPLLLSALPSFLCLVFKKC,0
1,DKLIGSCVWLAVNYTSNCNAECKRRGYKGGHCGSFLNVNCWCET,0
2,AVKDTYSCFIMRGKCRHECHDFEKPIGFCTKLNANCYM,0
3,GLPTCGETCFGGTCNTPGCTCDPWPVCTHN,1
4,ENCGRQAG,0


In [ ]:
acp_train_df, acp_train_features, acp_train_labels = extract_features_labels(acp_train_df)
acp_test_df, acp_test_features, acp_test_labels = extract_features_labels(acp_test_df)

One-hot encode

In [ ]:
acp_train_features

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,R,R,W,W,R,R,W,R,R,W,...,None,None,None,None,None,None,None,None,None,None
1,G,W,K,S,V,F,R,K,A,K,...,None,None,None,None,None,None,None,None,None,None
2,A,L,W,K,T,M,L,K,K,L,...,None,None,None,None,None,None,None,None,None,None
3,G,L,F,D,V,I,K,K,V,A,...,None,None,None,None,None,None,None,None,None,None
4,V,A,K,L,L,A,K,L,A,K,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,L,L,K,L,L,K,W,L,L,K,...,None,None,None,None,None,None,None,None,None,None
1374,G,F,K,D,W,I,K,G,A,A,...,None,None,None,None,None,None,None,None,None,None
1375,E,S,E,F,D,R,Q,E,Y,E,...,D,W,S,K,Y,D,N,Q,E,None
1376,F,I,S,A,I,A,S,F,L,G,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
acp_test_features

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,F,L,P,L,L,L,S,A,L,P,...,None,None,None,None,None,None,None,None,None,None
1,D,K,L,I,G,S,C,V,W,L,...,W,C,E,T,None,None,None,None,None,None
2,A,V,K,D,T,Y,S,C,F,I,...,None,None,None,None,None,None,None,None,None,None
3,G,L,P,T,C,G,E,T,C,F,...,None,None,None,None,None,None,None,None,None,None
4,E,N,C,G,R,Q,A,G,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,L,L,K,K,L,L,K,W,L,K,...,None,None,None,None,None,None,None,None,None,None
340,G,L,L,L,D,T,L,K,G,A,...,None,None,None,None,None,None,None,None,None,None
341,F,W,G,A,L,A,K,G,A,L,...,None,None,None,None,None,None,None,None,None,None
342,G,C,L,E,F,W,W,K,C,N,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

ohe = OneHotEncoder(handle_unknown='ignore')

x_train = ohe.fit_transform(acp_train_features)
y_train = acp_train_labels['label_acp'].to_numpy()

In [ ]:
x_test = ohe.transform(acp_test_features)
y_test = acp_test_labels['label_acp'].to_numpy()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1378, 1005), (1378,), (344, 1005), (344,))

**Logistic Regression**

In [ ]:
lr = LogisticRegression(max_iter=10000)
lr.fit(x_train, y_train)


LogisticRegression(max_iter=10000)

**SVM**

In [ ]:
svc = LinearSVC(max_iter=10000)
svc.fit(x_train, y_train)


LinearSVC(max_iter=10000)

**XGBoost**

Examples - https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn/notebook

In [ ]:
from xgboost import XGBClassifier

/Users/Vinod/opt/anaconda3/envs/peptide/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [ ]:
xgb = XGBClassifier()
xgb.fit(x_train, y_train)

/Users/Vinod/opt/anaconda3/envs/peptide/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:54:17] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
lr_preds = lr.predict(x_test)
svc_preds = svc.predict(x_test)
xgb_preds = xgb.predict(x_test)

In [ ]:
from sklearn.metrics import *
scores = []
for preds in [lr_preds, svc_preds, xgb_preds]:
    scores.append([accuracy_score(y_test, preds), f1_score(y_test, preds), precision_score(y_test, preds)])


NameError: name 'lr_preds' is not defined

In [ ]:
pd.DataFrame(scores, columns=['acc', 'f1', 'precision'], index=['lr', 'svc', 'xgb'])

,acc,f1,precision
lr,0.747093,0.757660,0.727273
svc,0.697674,0.704545,0.688889
xgb,0.755814,0.770492,0.726804


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:

import string


amino_acids = list(string.ascii_uppercase)

In [ ]:
amino_acids.append('PAD')

In [ ]:
amino_acids

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'PAD']

In [ ]:
acp_train_features.fillna('PAD')

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,R,R,W,W,R,R,W,R,R,W,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
1,G,W,K,S,V,F,R,K,A,K,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
2,A,L,W,K,T,M,L,K,K,L,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
3,G,L,F,D,V,I,K,K,V,A,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
4,V,A,K,L,L,A,K,L,A,K,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,L,L,K,L,L,K,W,L,L,K,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
1374,G,F,K,D,W,I,K,G,A,A,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD
1375,E,S,E,F,D,R,Q,E,Y,E,...,D,W,S,K,Y,D,N,Q,E,PAD
1376,F,I,S,A,I,A,S,F,L,G,...,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD,PAD


In [ ]:
ohe2 = OneHotEncoder(categories = [amino_acids] * acp_train_features.shape[1])

In [ ]:
X_train = ohe2.fit_transform(acp_train_features.fillna('PAD'))
y_train = acp_train_labels['label_acp'].to_numpy()

In [ ]:
X_test = ohe2.transform(acp_test_features.fillna('PAD'))
y_test = acp_test_labels['label_acp'].to_numpy()

In [ ]:
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)


LogisticRegression(max_iter=10000)

In [ ]:
svc = LinearSVC(max_iter=10000)
svc.fit(X_train, y_train)


LinearSVC(max_iter=10000)

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

/Users/Vinod/opt/anaconda3/envs/peptide/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
lr_preds = lr.predict(X_test)
svc_preds = svc.predict(X_test)
xgb_preds = xgb.predict(X_test)

In [ ]:
from sklearn.metrics import *
scores = []
for preds in [lr_preds, svc_preds, xgb_preds]:
    scores.append([accuracy_score(y_test, preds), f1_score(y_test, preds), precision_score(y_test, preds)])


In [ ]:
pd.DataFrame(scores, columns=['acc', 'f1', 'precision'], index=['lr', 'svc', 'xgb'])

,acc,f1,precision
lr,0.747093,0.757660,0.727273
svc,0.697674,0.704545,0.688889
xgb,0.755814,0.770492,0.726804


In [ ]:
pd.DataFrame(scores, columns=['acc', 'f1', 'precision'], index=['lr', 'svc', 'xgb'])

,acc,f1,precision
lr,0.747093,0.757660,0.727273
svc,0.697674,0.704545,0.688889
xgb,0.755814,0.770492,0.726804
